In [2]:
import pyterrier as pt
pt.init()



PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [9]:
dataset = pt.get_dataset('irds:beir/trec-covid')
index = pt.IndexFactory.of('../indices/cord19')
print(index.getCollectionStatistics().toString())

Number of documents: 192509
Number of terms: 171124
Number of postings: 13188177
Number of fields: 4
Number of tokens: 20627415
Field names: [title, doi, date, abstract]
Positions:   false



In [7]:
# get the arguement query 
import pandas as pd 
querys = pd.read_csv("argument_query.txt",sep = "\t", names = ["qid","query"])
print(querys)
qrels = pd.read_csv("qrels.txt", sep = "\t", names = ["qid","docno","label", "iteration"])

qrels = qrels.astype({"docno": str, "qid": str})
print(qrels)
querys = querys.astype({"qid": str})



     qid                                              query
0     51              what is the origin of the coronavirus
1     52                where did the coronavirus come from
2     53                     how did the coronavirus emerge
3     54        what caused the outbreak of the coronavirus
4     55  when and how was the coronavirus first identified
..   ...                                                ...
245  296  what is an mrna vaccine and how does it work a...
246  297  what are the differences between mrna vaccines...
247  298  what are the potential side effects of mrna co...
248  299  how effective are mrna covid vaccines in preve...
249  300  what is the recommended dosage and schedule fo...

[250 rows x 2 columns]
        qid     docno  label  iteration
0        51  005b2j4b      2        4.5
1        51  00fmeepz      1        4.0
2        51  010vptx3      2        0.5
3        51  0194oljo      1        2.5
4        51  021q9884      1        4.0
...     ...     

In [10]:
new_query = pd.concat([dataset.get_topics("query"), querys], ignore_index=True)
print(new_query)

     qid                                              query
0      1                                 coronavirus origin
1      2            coronavirus response to weather changes
2      3                               coronavirus immunity
3      4             how do people die from the coronavirus
4      5                          animal models of covid 19
..   ...                                                ...
295  296  what is an mrna vaccine and how does it work a...
296  297  what are the differences between mrna vaccines...
297  298  what are the potential side effects of mrna co...
298  299  how effective are mrna covid vaccines in preve...
299  300  what is the recommended dosage and schedule fo...

[300 rows x 2 columns]


In [13]:
new_query_sample = new_query.sample(frac=1).reset_index(drop=True)
print(new_query_sample)

     qid                                              query
0    287  what factors should be considered when decidin...
1     13                        how does coronavirus spread
2    155    how has coronavirus mortality changed over time
3    273  what resources are available for those struggl...
4    117  how do coronavirus super spreaders contribute ...
..   ...                                                ...
295  233  how does the phylogenetic structure of sars co...
296   37                   sars cov 2 phylogenetic analysis
297  149    can ace inhibitors be used to treat coronavirus
298  103  how are hospitals prioritizing patients during...
299  219  are there any long term health effects associa...

[300 rows x 2 columns]


In [24]:
train_query = new_query_sample[:int(len(new_query_sample)*0.8)]
test_query = new_query_sample[int(len(new_query_sample)*0.8):]
print(len(train_query))
print(len(test_query))

train_query.to_csv("train_query.txt", sep = "\t", index = False)
test_query.to_csv("test_query.txt", sep = "\t", index = False)

240
60


In [18]:
new_qrels = pd.concat([dataset.get_qrels(), qrels],ignore_index=True)
print(new_qrels)

        qid     docno  label iteration
0         1  005b2j4b      2         0
1         1  00fmeepz      1         0
2         1  g7dhmyyo      2         0
3         1  0194oljo      1         0
4         1  021q9884      1         0
...     ...       ...    ...       ...
412921  300  zvop8bxh      2       5.0
412922  300  zwf26o63      1       5.0
412923  300  zwsvlnwe      0       5.0
412924  300  zxr01yln      1       5.0
412925  300  zz8wvos9      1       5.0

[412926 rows x 4 columns]


In [26]:
qrels_train = new_qrels[new_qrels["qid"].isin(train_query["qid"])]
qrels_test = new_qrels[new_qrels["qid"].isin(test_query["qid"])]
print(qrels_train)
print(qrels_test)
qrels_train.to_csv("qrels_train.txt", sep = "\t", index = False)
qrels_test.to_csv("qrels_test.txt", sep = "\t", index = False)

        qid     docno  label iteration
0         1  005b2j4b      2         0
1         1  00fmeepz      1         0
2         1  g7dhmyyo      2         0
3         1  0194oljo      1         0
4         1  021q9884      1         0
...     ...       ...    ...       ...
412921  300  zvop8bxh      2       5.0
412922  300  zwf26o63      1       5.0
412923  300  zwsvlnwe      0       5.0
412924  300  zxr01yln      1       5.0
412925  300  zz8wvos9      1       5.0

[334389 rows x 4 columns]
        qid     docno  label iteration
1565      2  01goni72      2         0
1566      2  01yc7lzk      0         0
1567      2  02cy1s8x      0         0
1568      2  02f0opkr      0         0
1569      2  vprjbzw8      2         0
...     ...       ...    ...       ...
407383  294  zwbc7nnn      0       5.0
407384  294  zwsvlnwe      0       5.0
407385  294  zxx7tikz      0       5.0
407386  294  zy9lb7d9      1       5.0
407387  294  zzkgjpnq      0       5.0

[78537 rows x 4 columns]


# Conducting the bm25 Search to the train and test

In [21]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pt.Experiment(
    [bm25],
    train_query,
    qrels_train,
    eval_metrics=["map", "ndcg_cut.10", "ndcg_cut.20", "ndcg_cut.30", "ndcg_cut.100", "P.10", "P.20", "P.30", "P.100"],
)

,name,map,ndcg_cut.10,ndcg_cut.20,ndcg_cut.30,ndcg_cut.100,P.10,P.20,P.30,P.100
0,BR(BM25),0.131922,0.435345,0.406701,0.389017,0.332297,0.478333,0.448542,0.425,0.355792


In [23]:
pt.Experiment(
    [bm25],
    test_query,
    qrels_test,
    eval_metrics=["map", "ndcg_cut.10", "ndcg_cut.20", "ndcg_cut.30", "ndcg_cut.100", "P.10", "P.20", "P.30", "P.100"],
)

,name,map,ndcg_cut.10,ndcg_cut.20,ndcg_cut.30,ndcg_cut.100,P.10,P.20,P.30,P.100
0,BR(BM25),0.158893,0.44499,0.428813,0.421533,0.365465,0.505,0.480833,0.472778,0.401
